In [1]:
import requests

import urllib.request
import json 

import numpy as np

In [20]:
# Пример данных
data = ( 
'Home Mortgage',
    2396375.0,
    '6 years',
    0.0,
    9.0,
    9.5,
    335280.0,
    1.0,
    8.0,
    1.0,
    'debt consolidation',
    'Short Term',
    621544.0,
    139859.0,
    33350.0,
    71440.0
)

In [21]:
# формируем запрос
def send_json(x):
    (home_ownership, annual_income, years_job, tax_liens, accounts, years_history, max_credit, n_problems, last_delinquent, bankruptcies, purpose, 
    term, current_loan_amount, current_credit_balance, monthly_debt, credit_score) = x

    body = {'Home Ownership' : home_ownership,
    'Annual Income' : annual_income,
    'Years in current job' : years_job,
    'Tax Liens' : tax_liens,
    'Number of Open Accounts' : accounts,
    'Years of Credit History' : years_history,
    'Maximum Open Credit' : max_credit,
    'Number of Credit Problems' : n_problems,
    'Months since last delinquent' : last_delinquent,
    'Bankruptcies' : bankruptcies,
    'Purpose' : purpose,
    'Term' : term,
    'Current Loan Amount' : current_loan_amount ,
    'Current Credit Balance' : current_credit_balance,
    'Monthly Debt' : monthly_debt,
    'Credit Score' : credit_score}
    
    myurl = 'http://127.0.0.1:5000' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [22]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

предсказание 0.7353031576952568


In [10]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [11]:
response = send_json(X_test[['Home Ownership',
 'Annual Income',
 'Years in current job',
 'Tax Liens',
 'Number of Open Accounts',
 'Years of Credit History',
 'Maximum Open Credit',
 'Number of Credit Problems',
 'Months since last delinquent',
 'Bankruptcies',
 'Purpose',
 'Term',
 'Current Loan Amount',
 'Current Credit Balance',
 'Monthly Debt',
 'Credit Score']].iloc[0,:])


In [12]:
response

0.5517057934289515

In [13]:
X_test[['Home Ownership',
 'Annual Income',
 'Years in current job',
 'Tax Liens',
 'Number of Open Accounts',
 'Years of Credit History',
 'Maximum Open Credit',
 'Number of Credit Problems',
 'Months since last delinquent',
 'Bankruptcies',
 'Purpose',
 'Term',
 'Current Loan Amount',
 'Current Credit Balance',
 'Monthly Debt',
 'Credit Score']].head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score
0,Rent,658217.0,10+ years,0.0,9.0,10.9,150260.0,0.0,3.0,0.0,debt consolidation,Short Term,139370.0,53200.0,10970.0,733.0
1,Home Mortgage,1028242.0,7 years,0.0,6.0,13.9,123596.0,0.0,43.0,0.0,debt consolidation,Short Term,99999999.0,56791.0,9254.0,734.0
2,Rent,1045323.0,10+ years,0.0,4.0,16.3,299090.0,0.0,33.0,0.0,debt consolidation,Long Term,440132.0,181070.0,11847.0,732.0
3,Rent,858173.0,< 1 year,0.0,7.0,10.7,213994.0,0.0,30.0,0.0,debt consolidation,Short Term,237622.0,92226.0,9225.0,721.0
4,Rent,1747772.0,4 years,0.0,13.0,24.8,189398.0,0.0,38.0,0.0,debt consolidation,Long Term,753522.0,161937.0,33499.0,667.0


Сделаем N запросов и оценим время

In [23]:
N = 30

In [24]:
%%time
predictions = X_test[
                     ['Home Ownership',
 'Annual Income',
 'Years in current job',
 'Tax Liens',
 'Number of Open Accounts',
 'Years of Credit History',
 'Maximum Open Credit',
 'Number of Credit Problems',
 'Months since last delinquent',
 'Bankruptcies',
 'Purpose',
 'Term',
 'Current Loan Amount',
 'Current Credit Balance',
 'Monthly Debt',
 'Credit Score']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: total: 312 ms
Wall time: 10.2 s


In [25]:
predictions.values[:7]

array([0.55170579, 0.31195766, 0.5280765 , 0.55482798, 0.54222049,
       0.52018987, 0.49688889])

Посчитаем метрику

In [27]:
y_test.head(7)

,Credit Default
0,1
1,0
2,0
3,0
4,1
5,1
6,0


In [18]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.5395847184062018, F-Score=0.645, Precision=0.556, Recall=0.769


In [19]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.8087318087318087